In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time

In [2]:
def login(user, password):
    
    #object of FirefoxOptions
    options = webdriver.FirefoxOptions()
    options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'

    driver = webdriver.Firefox(options=options)
    a = ActionChains(driver)

    driver.get("https://linkedin.com/")
    # Login
    time.sleep(2)
    
    input_user = driver.find_element(By.ID, "session_key")
    input_user.send_keys(user)
    input_password = driver.find_element(By.ID, "session_password")
    input_password.send_keys(password)

    input_submit = driver.find_element(By.CLASS_NAME, "sign-in-form__submit-button")
    time.sleep(2)
    input_submit.click()
    time.sleep(30)
    
    return driver

In [32]:
def send(driver):
    # Find send
    button2 = driver.find_elements(By.CLASS_NAME, "artdeco-button--2")
    button2 = [x for x in button2 if x.get_attribute('innerHTML').find('Send')>0]

    if len(button2) == 0:  # because additional options then skip
        driver.find_element(By.CLASS_NAME, "artdeco-modal-overlay").click()
        return False
    else:
        
        # check if verify email
        verify = driver.find_elements(By.CLASS_NAME, 'artdeco-modal__content')
        if len(verify) > 0:
            if verify[0].get_attribute('innerHTML').find('To verify this member knows you') > 0:
                driver.find_element(By.CLASS_NAME, "artdeco-modal-overlay").click()
                return False 
        else:
            
            button2[0].click()
            return True

In [33]:
def connect(driver):
    n = 0
    
    #find all connect
    button = driver.find_elements(By.CLASS_NAME, "artdeco-button--2")
    button = [x for x in button if x.get_attribute('innerHTML').find('Connect')>0]

    for i in range(len(button)):
        button[i].click()
        time.sleep(1)

        if send(driver):
            n += 1
            
        time.sleep(1)
        
    return n

In [34]:
def follow(driver):
    
    n = 0
    
    follow_items = driver.find_elements(By.CLASS_NAME, "entity-result__item")
    follow_items = [x for x in follow_items if x.get_attribute('innerHTML').find('Follow')>0]

    gethref = lambda x: x.find_element(By.CLASS_NAME, "app-aware-link").get_property('href').split('?')[0]

    if len(follow_items) > 0:
        follow_items = [gethref(x) for x in follow_items]


        for follow in follow_items:

            # Open a new window
            driver.execute_script("window.open('');")
            # Switch to the new window
            driver.switch_to.window(driver.window_handles[1])
            driver.get(follow)
            time.sleep(10)

            # open dropdown menu
            more = driver.find_elements(By.CLASS_NAME, "artdeco-dropdown__trigger")
            more[2].click()

            button = driver.find_elements(By.CLASS_NAME, "artdeco-dropdown__item")
            button = [x for x in button if x.get_attribute('innerHTML').find('Connect')>0]

            button[1].click()

            send(driver)

            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            
            n += 1
    return n

In [35]:
def url_search(geo=['MX'], keywords='data', page=1):
    geo_dict = {'MX': "103323778", 'CO': "100876405", 'CL': "104621616"}
    geo_str = [geo_dict[x] for x in geo]
    
    url = f'https://www.linkedin.com/search/results/people/?geoUrn={str(geo_str)}&keywords={keywords}&origin=FACETED_SEARCH&page={page}'
    return url.replace("'", '"')

In [7]:
from credentials import user, password

In [8]:
driver = login(user, password)

In [9]:
n = 0

In [36]:
keywords = 'data'
geo = ['CL']

In [ ]:

for page in range(1, 3):
    
    print(f'page {page}')
    driver.get(url_search(geo=geo, keywords=keywords, page=page))
    time.sleep(20)
    
    n1 = connect(driver)
    n += n1
    
    n2 = follow(driver)
    n += n2
    print(f'page {page} - connect {n1} - follow {n2}')
    
print(n)

page 1
